<a href="https://colab.research.google.com/github/koklengyeo/aviation/blob/main/steel_procurement_v1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PREAMBLE

In [19]:
#FOR FUTURE REFERENCE
#DOWNGRADE PYTHON TO VERSION USED DURING DEVELOPMENT
#!apt-get update -y
#!apt-get install python3.10 python3.10-distutils
#!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.10 1
#!update-alternatives --config python3
#!apt-get install python3-pip
#!python3 -m pip install --upgrade pip --user

In [20]:
#FOR FUTURE REFERENCE
#DOWNGRADE PACKAGES TO VERSION USED DURING DEVELOPMENT
#!pip install IPython==7.34.0
#!pip install ipywidgets==7.7.1
#!pip install matplotlib==3.7.1
#!pip install numpy==1.25.2
#!pip install pandas==2.0.3

In [21]:
#IMPORT PACKAGES
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
from IPython.display import *
from ipywidgets import *

# USER INPUT FORM (WIDGETS)

In [22]:
#DEFINE WIDGETS USED TO CAPTURE USER INPUTS

#BASE YEAR
#MUST BETWEEN 2015 AND PRESENT YEAR
by_widget=widgets.BoundedIntText(
    value=datetime.now().year,
    min=2015,
    max=datetime.now().year,
    step=1,
    description="Base Year",
    disabled=False)

#TARGET YEAR
#MUST BE BETWEEN (PRESENT YEAR+5) AND 2050
ty_widget=widgets.BoundedIntText(
    value=datetime.now().year+5,
    min=datetime.now().year,
    max=2050,
    step=1,
    description="Target Year",
    disabled=False)

#BASE YEAR PURCHASED STEEL EMISSIONS IN TCO2E
#MUST BE >=0
pse_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Purchased Steel Emissions in tCO2e",
    disabled=False)

#BASE YEAR PURCHASED STEEL IN TONNES
#MUST BE >=0
by_psa_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Purchased Steel in Tonnes",
    disabled=False)

#TARGET YEAR PURCHASED STEEL IN TONNES
#MUST BE >=0
ty_psa_widget=widgets.BoundedIntText(
    value=0,
    min=0,
    max=1e99,
    step=1,
    description="Purchased Steel in Tonnes",
    disabled=False)

#INVISIBLE BOOLEAN WIDGET TO REGISTER FIRST CLICK OF "CALCULATE TARGET" BUTTON
fc_widget=widgets.Valid(
    value=False,
    layout={"display":"none"})

In [23]:
#DEFINE WIDGETS USED TO DISPLAY TEXT

overall_header=widgets.HTML(
    value="<h1>TARGET SETTING FOR STEEL PROCUREMENT</h1>\
    <h3>Version 1.0 (Jul-23)</h3>This is intended for companies procuring \
    steel to set targets on scope 3 category 1 'Purchased Goods and Services' \
    emissions. Targets are aligned with a 1.5&deg;C scenario.")
basic_info_header=widgets.HTML(
    value="<h2>1. Basic Information</h2>Please indicate the base year and \
    target year. Target must cover a minimum of 5 years and a maximum of 10 \
    years from the year it is submitted to the SBTi for validation.")
by_emissions_header=widgets.HTML(
    value="<h2>2. Base Year Emissions</h2>")
by_activity_header=widgets.HTML(
    value="<h2>3. Base Year Activity</h2>Please provide the amount of steel \
    purchased in the base year.")
ty_activity_header=widgets.HTML(
    value="<h2>4. Target Year Activity</h2>Please provide the expected amount\
    of steel to be purchased in the target year.")
ty_activity_footer=widgets.HTML(
    value="Please ensure that all provided information is as complete and \
    accurate as possible. Click the button below to calculate the target.")
results_header=widgets.HTML(
    value="<h2>5. Target Modelling Results</h2>")

#ERROR MESSAGES
no_error=widgets.HTML()
pse_error=widgets.HTML(
    value="<p style='color:red;'>Purchased Steel Emissions in tCO2e must be \
      provided.</p>")
psa_error=widgets.HTML(
    value="<p style='color:red;'>Purchased Steel in Tonnes must be \
      provided.</p>")
user_inputs_error=widgets.HTML(
    value="<p style='color:red;'>Information provided incomplete and/or \
      invalid. Please check and try again.</p>")
no_target_error=widgets.HTML(
    value="<p style='color:red;'>No target calculated. The base year intensity \
      is already below the 2050 level in the 1.5&deg;C scenario.</p>")

In [24]:
#WIDGET LAYOUT & STYLE

#YEAR WIDGETS
layout={"width":"700px"}
style={"description_width":"600px"}
by_widget.layout=layout
by_widget.style=style
ty_widget.layout=layout
ty_widget.style=style

#EMISSIONS WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
pse_widget.layout=layout
pse_widget.style=style

#ACTIVITY WIDGETS
layout={"width":"800px"}
style={"description_width":"600px"}
by_psa_widget.layout=layout
by_psa_widget.style=style
ty_psa_widget.layout=layout
ty_psa_widget.style=style

#TEXT WIDGETS
layout={"width":"1100px"}
overall_header.layout=layout
basic_info_header.layout=layout
by_emissions_header.layout=layout
by_activity_header.layout=layout
ty_activity_header.layout=layout
ty_activity_footer.layout=layout
results_header.layout=layout
pse_error.layout=layout
psa_error.layout=layout
user_inputs_error.layout=layout
no_target_error.layout=layout

# USER INPUT FORM (INTERACTIVITY)

In [25]:
#USER INPUT FORM - BASIC INFORMATION

def basic_info_check(by,
                     ty,
                     fc):
  return True
basic_info_input=interactive(
    basic_info_check,
    by=by_widget,
    ty=ty_widget,
    fc=fc_widget)

In [26]:
#USER INPUT FORM - BASE YEAR EMISSIONS

def by_emissions_check(pse,
                       fc):
  by_emissions_valid=True
  update_display(no_error,display_id="by_emissions_check")
  #PSE - MUST BE PROVIDED
  if pse==0:
    by_emissions_valid=False
    if fc:
      display(pse_error,display_id="by_emissions_check")
  return by_emissions_valid
by_emissions_input=interactive(
    by_emissions_check,
    pse=pse_widget,
    fc=fc_widget)

In [27]:
#USER INPUT FORM - BASE YEAR ACTIVITY

def by_activity_check(by_psa,
                      fc):
  by_activity_valid=True
  update_display(no_error,display_id="by_activity_check")
  #BY_PSA - MUST BE PROVIDED
  if by_psa==0:
    by_activity_valid=False
    if fc:
      display(psa_error,display_id="by_activity_check")
  return by_activity_valid
by_activity_input=interactive(
    by_activity_check,
    by_psa=by_psa_widget,
    fc=fc_widget)

In [28]:
#USER INPUT FORM - TARGET YEAR ACTIVITY

def ty_activity_check(ty_psa,
                      fc):
  ty_activity_valid=True
  update_display(no_error,display_id="ty_activity_check")
  #TY_PSA - MUST BE PROVIDED
  if ty_psa==0:
    ty_activity_valid=False
    if fc:
      display(psa_error,display_id="ty_activity_check")
  return ty_activity_valid
ty_activity_input=interactive(
    ty_activity_check,
    ty_psa=ty_psa_widget,
    fc=fc_widget)

# TARGET MODEL (DATA & PARAMETERS)

In [29]:
#IMPORT DATA

path="https://raw.githubusercontent.com/koklengyeo/migration/main/"

#IMPORT SECTOR DATA
steel=pd.read_csv(
    path+"steel.csv",
    decimal=",",
    index_col=0,
    sep=";")
steel.columns=steel.columns.map(int)

In [30]:
#DEFINE PARAMETERS

pm={#INTENSITY UNIT CONVERSION
    #INTENSITY=IUC*EMISSIONS/ACTIVITY (IN SECTOR DATA UNITS)
    "iuc":1,
    #YEAR AXIS
    "year":pd.Series(range(2015,2051)).set_axis(range(2015,2051))}

# TARGET MODEL (FUNCTIONS)

In [31]:
#FUNCTION - CHECK IF USER INPUTS SATISFY IMPOSED CONDITIONS

def user_inputs_check():
  if not basic_info_input.result \
  or not by_emissions_input.result \
  or not by_activity_input.result \
  or not ty_activity_input.result:
    user_inputs_valid=False
  else:
    user_inputs_valid=True
  return user_inputs_valid

In [32]:
#FUNCTION - CAPTURE USER INPUTS INTO DICTIONARY

def user_inputs_capture():
  return {"by":basic_info_input.children[0].value,
          "ty":basic_info_input.children[1].value,
          "pse":by_emissions_input.children[0].value,
          "by_psa":by_activity_input.children[0].value,
          "ty_psa":ty_activity_input.children[0].value}

In [33]:
#FUNCTION - SECTOR DECARBONIZATION APPROACH

def sda(
    #SECTOR OR SECTOR SEGMENT NAME (IN SECTOR DATA)
    name=None,
    #BASE YEAR
    by=None,
    #TARGET YEAR
    ty=None,
    #BASE YEAR EMISSIONS (IN SAME UNIT AS SECTOR DATA)
    pse=None,
    #ACTIVITY FORECAST (IN SAME UNIT AS SECTOR DATA)
    af=None,
    #SECTOR DATA
    sector_data=None,
    #INTENSITY UNIT CONVERSION
    #(SECTOR INTENSITY UNIT)*(SECTOR ACTIVITY UNIT)/(SECTOR EMISSIONS UNIT)
    iuc=None):

  #SECTOR EMISSIONS, ACTIVITY & INTENSITY
  output=sector_data.loc[sector_data.index.str.contains(name)]
  for i in ["Emissions","Activity","Intensity"]:
    _=output.index[output.index.str.contains(name+" "+i)].values[0]
    output=output.rename(index={_:"Sector "+i})

  #COMPANY ACTIVITY
  output.loc["Company Activity"]=af

  #SDA (EQUATION 6 IN SDA REPORT)
  #SI TERM
  si=output.loc["Sector Intensity",2050]
  #P TERM
  p=output.loc["Sector Intensity"]-si
  p/=output.loc["Sector Intensity",by]-si
  #D TERM
  if output.loc["Company Activity",by]!=0:
    d=iuc*pse/output.loc["Company Activity",by]-si
  else:
    d=-si
  #BASE YEAR INTENSITY > CONVERGENCE INTENSITY
  if d>0:
    set_target=True
  #BASE YEAR INTENSITY <= CONVERGENCE INTENSITY
  else:
    set_target=False
  #M TERM
  m=output.loc["Company Activity",by]/output.loc["Sector Activity",by]
  m/=output.loc["Company Activity"]/output.loc["Sector Activity"]
  m[m>1]=1

  #COMPANY INTENSITY
  if set_target:
    output.loc["Company Intensity"]=d*p*m+si
  else:
    output.loc["Company Intensity"]=0
  output.loc["Company Intensity"]=output.loc["Company Intensity"].fillna(0)

  #COMPANY EMISSIONS
  output.loc["Company Emissions"]=\
  output.loc["Company Intensity"]*output.loc["Company Activity"]/iuc
  output.loc["Company Emissions"]=output.loc["Company Emissions"].fillna(0)

  #MASK YEAR<BASE YEAR
  output.loc[output.index.str.contains("Company"),output.columns<by]=np.nan

  #SORT
  output=output.sort_index()

  return output

In [34]:
#FUNCTION - TARGET MODEL

def target_model():

  #ACTIVITY FORECAST
  af=(pm["year"]-ip["by"])*(ip["ty_psa"]-ip["by_psa"])/(ip["ty"]-ip["by"])
  af+=ip["by_psa"]

  #SECTOR INTENSITY
  sector_data=steel.copy(deep=True)
  sector_data.loc["Total Intensity (tCO2e/t)"]=\
  sector_data.loc["Total Emissions (tCO2e)"]
  sector_data.loc["Total Intensity (tCO2e/t)"]/=\
  sector_data.loc["Total Activity (t)"]

  #SDA TARGET
  output=sda(name="Total",
             by=ip["by"],
             ty=ip["ty"],
             pse=ip["pse"],
             af=af,
             sector_data=sector_data,
             iuc=pm["iuc"])

  return output

# RESULTS (FUNCTIONS)

In [35]:
#FUNCTION - TARGET TABLE

def target_table():
  #FETCH BASE YEAR & TARGET YEAR DATA
  table=target.loc["Company Intensity",[ip["by"],ip["ty"]]]
  table=table.to_frame().transpose()

  #% REDUCTION
  table["% Reduction ("+str(ip["by"])+"-"+str(ip["ty"])+")"]=\
  -1e2*(table[ip["ty"]]/table[ip["by"]]-1)

  #RENAME ROWS
  table.index=["Company Core Boundary Intensity (tCO2e/t)"]

  #RENAME COLUMNS
  table=table.rename({ip["by"]:"Base Year ("+str(ip["by"])+")",
                      ip["ty"]:"Target Year ("+str(ip["ty"])+")"},
                     axis=1)

  #ROUNDING
  table.iloc[:,:2]=table.iloc[:,:2].apply(lambda x:round(x,2))
  table.iloc[:,2:]=table.iloc[:,2:].apply(lambda x:round(x))

  #PERCENTAGE SIGN
  table.iloc[:,2:]=table.iloc[:,2:].astype(int).astype(str)+"%"

  #FORMAT DISPLAY
  table=table.style
  #NUMBER OF DECIMAL PLACES
  table=table.format(precision=2)
  #CENTRE ALIGN
  table=table.set_properties(**{"text-align":"center"})
  #COLUMN HEADING - MAX WIDTH 100PX & CENTRE ALIGN
  #ROW HEADING - RIGHT ALIGN
  table=table.set_table_styles(
      [dict(selector=".col_heading",
            props=[("max-width","100px"),("text-align","center")]),
       dict(selector=".row_heading",
            props=[("text-align","right")])])

  display(table)

# FINAL PRODUCT

In [36]:
#BUTTON WIDGET
calculate_target_button=widgets.Button(description="Calculate Target")

#OUTPUT WIDGET
results=widgets.Output()

#SPACER WIDGET
spacer=widgets.HTML(value="<br>")

#TARGET MODEL & RESULTS
@results.capture(clear_output=True)
def target_model_results(_):
  #REGISTER FIRST CLICK
  fc_widget.value=True
  #USER INPUTS DO NOT MEET IMPOSED CONDITIONS
  if not user_inputs_check():
    #ERROR MESSAGE
    display(user_inputs_error)
  #USER INPUTS MEET IMPOSED CONDITIONS
  else:
    #TARGET MODEL
    global ip,target
    ip=user_inputs_capture()
    target=target_model()
    #CHECK IF MODEL RETURNED TARGET
    if target.loc["Company Intensity",ip["by"]]>0:
      no_target=False
    else:
      no_target=True
    #RESULTS
    if no_target:
      display(no_target_error)
    else:
      display(results_header)
      target_table()

#LINK BUTTON TO FUNCTION
calculate_target_button.on_click(target_model_results)

#DISPLAY
_=VBox([
    #USER INPUT FORM
    overall_header,
    basic_info_header,
    basic_info_input,
    by_emissions_header,
    by_emissions_input,
    by_activity_header,
    by_activity_input,
    ty_activity_header,
    ty_activity_input,
    ty_activity_footer,
    #TARGET MODEL & RESULTS
    calculate_target_button,
    results],
    layout={"width":"1110px"})
display(_)